### Combining programs properly

In [1]:
#import useful libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import pickle
import configuration
import general_statistics as stats
import figures as figs
import matplotlib.cm as cm
from matplotlib import colors
from scipy import signal
from scipy import stats as sstats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import scipy
cmap = cm.jet
import math
from sklearn.datasets import load_iris
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.cross_decomposition import CCA
from sklearn.model_selection import cross_val_score
import random
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA



import warnings
warnings.filterwarnings("ignore")
from analysis_functions import *


In [2]:
mouse = 56165      ### mouse number id
decoding_v = 1            ## decoding version, normaly equal to one
motion_correction_v = 20  ### 100 means everything was aligned, 200 means it was also equalized, 20 is for day wise
alignment_v = 3           ## alignment version, version 3 is for day wise
equalization_v = 0        ## equalization version
source_extraction_v = 1   ## source extraction version
component_evaluation_v = 1 ## component evaluation version

# here we will do analysis with no registration
# registration_v = 2        ## registration version
sf = 10                   ## sampling frequency of the original signal 
re_sf= 1                 ## value of resampling
session_now = 1
period = int(10 * sf / re_sf)
period_resting = period *50
N_SHUFFLINGS = 50
colorcodes = ['Greys','PuBu', 'YlOrRd','Reds','Blues','Oranges','Greens']


In [3]:
task,colapse_behaviour,object_fixed,fixed,labels,colornames = mouse_properties(mouse = mouse, session_now = session_now)

In [4]:
file_directory = os.environ['PROJECT_DIR'] + 'neural_analysis/data/calcium_activity_day_wise/'
timeline_file_dir = os.environ['PROJECT_DIR'] + 'neural_analysis/data/timeline/'
behaviour_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
behaviour_dir_parameters = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/ethogram_parameters/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
tracking_dir = os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/center_of_mass/' + f'{mouse}' + '/session_' + f'{session_now}' + '/'
objects_dir= os.environ['PROJECT_DIR'] + 'calcium_imaging_behaviour/data/object_positions/'
figure_path = os.environ['PROJECT_DIR'] +'neural_analysis/figures/trial_aligned_events/'

In [5]:
activity_list,timeline_list,behaviour_list,corners_list,parameters_time,parameters_list,parameters_list2, speed_list = load_data(mouse, session_now,decoding_v, motion_correction_v, alignment_v, equalization_v,source_extraction_v,component_evaluation_v, re_sf,file_directory, timeline_file_dir, behaviour_dir, behaviour_dir_parameters, tracking_dir, objects_dir)

LOADING TRIALS ACTIVITY AND CREATING LIST OF ACTIVITY, TRACKING AND BEHAVIOUR
(195, 32368)
(448, 30592)
(322, 31977)
(397, 30905)


In [6]:
data_transformation = transform_neural_data(activity_list, behaviour_list,parameters_time,parameters_list,parameters_list2)

In [7]:
create_task_behaviour(behaviour_list,colapse_behaviour,object_fixed)

REDIFINING BEHAVIOUR FOR DIFFERENT SESSIONS


In [8]:
print('CREATING CORNERS VECTORS VECTORS')
navigation_list, exploration_list = create_corners_occupation(behaviour_list, corners_list, speed_list)

CREATING CORNERS VECTORS VECTORS


In [11]:
print('CREATES LIST THAT SAVES ALL THE EVENTS IN A DAY AND CONTAINS ONSET OF VISITS')
### ethogram events list
events_etho, events_shuffle_1_etho,events_counter_etho,events_onset_etho = create_events_list(behaviour_list, N_SHUFFLINGS)
### navigation events list
events_nav, events_shuffle_1_nav,events_counter_nav,events_onset_nav = create_events_list(navigation_list, N_SHUFFLINGS)
### exploration events list
events_exp, events_shuffle_1_exp,events_counter_exp,events_onset_exp = create_events_list(exploration_list, N_SHUFFLINGS)


CREATES LIST THAT SAVES ALL THE EVENTS IN A DAY AND CONTAINS ONSET OF VISITS


In [12]:
print('NOW WE SEPARATE EVENTS TYPES ACCORDING TO CORNER/OBJECT VISIT')
## compute events duration for type of event. Here we create lists that we can access to know the duration of each ID event
## we will use this to balance the dataset
# id_for ethogram object exploration
id_target_etho = [3,4]
events_duration_etho, total_duration_etho,number_of_events_etho,events_id_etho = create_id_events(events_etho, events_counter_etho,events_onset_etho,id_target_etho)

# id for corners navigation
id_target_corners = [1,2,3,4]
events_duration_nav, total_duration_nav,number_of_events_nav,events_id_nav = create_id_events(events_nav, events_counter_nav,events_onset_nav,id_target_corners)

# id for corners navigation with objects
id_target_exp = [1,2,3,4]
events_duration_exp, total_duration_exp,number_of_events_exp,events_id_exp = create_id_events(events_exp, events_counter_exp,events_onset_exp,id_target_exp)

NOW WE SEPARATE EVENTS TYPES ACCORDING TO CORNER/OBJECT VISIT


NameError: name 'behaviour_list' is not defined

In [ ]:
print('BALANCING TO THE LOWER NUMBER OF VISITS: ethogram')
events_number_etho, events_etho_b, events_etho_s_b = balancing_visits(number_of_events_etho,events_duration_etho,events_id_etho)
print('BALANCING TO THE LOWER NUMBER OF VISITS: corners navigation')
events_number_nav, events_nav_b, events_nav_s_b = balancing_visits(number_of_events_nav,events_duration_nav,events_id_nav)
print('BALANCING TO THE LOWER NUMBER OF VISITS: exploration navigation')
events_number_exp, events_exp_b, events_exp_s_b = balancing_visits(number_of_events_exp,events_duration_exp,events_id_exp)


In [ ]:
print('CREATE SHUFFLE LABELS THAT PRESERVE BALANCE')


def create_shuffling(events_day_list_1,events_counter_day_listevents_time_starts_day,number_of_events_list,events_id,events_day_list,N_SHUFFLINGS):
### create shuffled behavioural labels that preserve balance and temporal structure
### the shuffling will be in visits or events (so we shuffle the labels of the complete event)

for day in range(len(events_day_list_1)):
    events_shuffle = []
    events = np.array(events_day_list_1[day])
    events_counter = np.array(events_counter_day_list[day])
    events_time = np.array(events_time_starts_day[day])
    arg_min_target_time = np.argmin(number_of_events_list[day])
    n_events = number_of_events_list[day][arg_min_target_time]
    
    for target in range(len(events_id[day])):
        #print(target)
        #print(len(events_day_list[day]))
        #print(events_day_list[day][target])
        all_events = np.where(events == events_id[day][target])[0]
        #print(len(all_events))
        position_events = all_events[events_day_list[day][target]] # select the balanced data  
        events_shuffle.append(position_events)
    
    for j in range(N_SHUFFLINGS):
        counter_permutations = 0
        for i in range(n_events):
            permutation = np.random.permutation(len(events_id[day]))
            #print(permutation)
            counter_permutations +=1
            for index in range(len(events_id[day])):
                events_day_list_shuffle_1[day][j][events_shuffle[index][i]] = events_id[day][permutation[index]]
                
    return events_day_list_shuffle_1

In [ ]:
events_etho_shuffle = create_shuffling(events_day_list_1,events_counter_day_listevents_time_starts_day,number_of_events_list,events_id,events_day_list,N_SHUFFLINGS):